# University Of Skovde
## Data Science Master's Programme 
### Data Mining Project
<b>Author:Welemhret Welay Baraki</b> <br/>
<b>Email:a20welba@student.his.se</b> <br/>

### Supervisors:
1. <b>Huseyin Kusetogullari</b>
2. <b>Niclas Ståhl</b>
3. <b>Yurong Li(Alice) </b>

# Books Recommender System

The recommender system uses <b> Collaborative Filtering</b> and <b>Content-based Filtering</b> Approach. 
The Recommender System was implemented using python,and have four main parts:
### The four parts are :
1. <b>Memory based Recommendation</b> -particularly User-based Recommendation using the cosine similarity
2. <b>Model based Recommendation</b> using the Singualr Value Decomposition(SVD) Using the Supervised Model
3. <b>Content-based Recommendation </b>based on the Book details
4. <b>Hybrid Recommendation</b> based on the recommmendation results of the three models

In [2]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
from surprise import SVD 
from surprise import Dataset 
from surprise import accuracy 
from surprise.model_selection import train_test_split
from surprise import Reader
from surprise.model_selection import cross_validate
import warnings
warnings.filterwarnings('ignore')

In [3]:
Books=pd.read_csv("BX-CSV-Dump/Books.csv", sep=';', error_bad_lines=False,encoding="latin-1")
Books.columns =['ISBN', 'BookTitle', 'BookAuthor', 'YearOfPublication', 'Publisher', 'ImageURL-S', 'ImageURL-M', 'ImageURL-L']
Users=pd.read_csv("BX-CSV-Dump/Users.csv", sep=';', error_bad_lines=False,encoding="latin-1")
Users.columns =['UserID', 'Location', 'Age']
Ratings=pd.read_csv("BX-CSV-Dump/Ratings.csv", sep=';', error_bad_lines=False,encoding="latin-1")
Ratings.columns =['UserID', 'ISBN', 'BookRating']

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'


## Data Preprocessing
### Books Data
Droping irrelevant columns of the Books data set (ImageURL-S, ImageURL-M,ImageURL-L)

In [4]:
Books=Books.drop(["ImageURL-S", "ImageURL-M","ImageURL-L"],axis=1)

In [5]:
# There are incorrect entries of 'DK Publishing Inc' and 'Gallimard' in year of publication that needs to correct 
Books['YearOfPublication'].unique()
#Check their ISBN with the Year of Publication to make necessary Corrections
Books.loc[Books['YearOfPublication']=='DK Publishing Inc' ]
Books.loc[ Books['YearOfPublication']=='Gallimard']

#Correcting the values of the different columns of the books with ISBNs 078946697X,0789466953 and 2070426769
Books.loc[Books['ISBN']=='078946697X',"YearOfPublication"] = 2000 
Books.loc[Books['ISBN']=='078946697X',"BookAuthor"] ="Michael Teitelbaum"  
Books.loc[Books['ISBN']=='078946697X',"BookTitle"] = "DK Readers: Creating the X-Men, How It All Began (Level 4: Proficient Readers)" 
Books.loc[Books['ISBN']=='078946697X',"Publisher"] = "DK Publishing Inc" 

Books.loc[Books['ISBN']=='0789466953',"YearOfPublication"] = 2000 
Books.loc[Books['ISBN']=='0789466953',"BookAuthor"] ="James Buckley"  
Books.loc[Books['ISBN']=='0789466953',"BookTitle"] = "DK Readers: Creating the X-Men, How Comic Books Come to Life (Level 4: Proficient Readers)" 
Books.loc[Books['ISBN']=='0789466953',"Publisher"] = "DK Publishing Inc" 

Books.loc[Books['ISBN']=='2070426769',"YearOfPublication"] = 2003 
Books.loc[Books['ISBN']=='2070426769',"BookAuthor"] ="Jean-Marie Gustave Le ClÃ?Â©zio"  
Books.loc[Books['ISBN']=='2070426769',"BookTitle"] = "Peuple du ciel, suivi de 'Les Bergers" 
Books.loc[Books['ISBN']=='2070426769',"Publisher"] = "Gallimard" 

In [6]:
#Change the YearOfPublication data type to integer
Books.YearOfPublication=pd.to_numeric(Books['YearOfPublication'],errors="ignore")
# The year of Publication starts from 0 to 2050, Some values are unrealistic
# I am going to replace with mean
#Replace the nan and the years exceeding 2020  with the mean 
Books.YearOfPublication.fillna(round(Books.YearOfPublication.mean()),inplace=True)

Books[Books.YearOfPublication>=2020]=Books.YearOfPublication.mean()
#Checking the Publisher Column that there is null
Books.loc[Books.Publisher.isnull()]
#In the Publisher two null values found and I replace with "Unknown"
Books.loc[Books.ISBN=="193169656X", "Publisher"] ="Unknown"
Books.loc[Books.ISBN=="1931696993","Publisher"] ="Unknown"

### Users Data 
##### Exploring the Users Data set to make some corrections and understand the data

In [7]:
# Age column has a NaN and some very high values. 
# Ages Higher than 100 are replaced with mean of the Ages 
Users.loc[(Users.Age>100) | (Users.Age<7),"Age"] =round(Users.Age.mean())
Users.Age.fillna(round(Users.Age.mean()),inplace=True)
Users.Age=Users.Age.astype(np.int32)

### Ratings Dataset
##### Exploring and Preprocessing of the Ratings Dataset

In [8]:
#Extracting the ratings (Books Ratings excluding 0)
ExplicitRating=Ratings[Ratings['BookRating'] !=0 ]

#Filtering the Users that rate more than 1000 Books and Books that have count Greater than 500 for Processing Simplicity
counts1 = ExplicitRating['UserID'].value_counts()
ExplicitRating = ExplicitRating[ExplicitRating['UserID'].isin(counts1[counts1 >= 1000].index)]
counts = ExplicitRating['BookRating'].value_counts()
ExplicitRating = ExplicitRating[ExplicitRating['BookRating'].isin(counts[counts >=500].index)]

#Filtering Ratings,Users and Books  which are users not included
FilteredRating=ExplicitRating[ExplicitRating.ISBN.isin(Books['ISBN'])] 
FilteredRating=ExplicitRating[ExplicitRating.UserID.isin(Users['UserID'])]

FilteredUsers =Users[Users['UserID'].isin(FilteredRating['UserID'])]
FilteredBooks=Books[Books['ISBN'].isin(FilteredRating['ISBN'])]

### Building the Utility Matrix


In [9]:
MergedBooksRatings = pd.merge(left=FilteredRating,right=FilteredBooks,  
                              how='inner', left_on='ISBN', 
                              right_on='ISBN')

#Building the Utility Matrix form the Books Ratings using the Book ISBN and UserID
UtilityMatrix=MergedBooksRatings.pivot(index='UserID',columns='ISBN' ,values='BookRating')
UtilityMatrix

ISBN,0000913154,0001046438,000104687X,0001047213,0001047973,0001048082,0001053736,0001053744,0001944711,0002005018,...,9974643058,9992003766,9997406990,9997409728,9997411757,9997507002,999750805X,9997519086,9997555635,B0000DYXID
UserID,,,,,,,,,,,,,,,,,,,,,
11676,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,8.0,...,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0
23902,NaN,9.0,6.0,9.0,9.0,8.0,NaN,NaN,NaN,NaN,...,NaN,5.0,7.0,8.0,7.0,NaN,6.0,5.0,8.0,NaN
76499,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98391,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
153662,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
171118,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
189835,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN
235105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### User-Based Recommendation  


In [10]:
# Calculating the Centered Cosine Similarity
#### Step-1- Normalize the Ratings of the Utility Matrix by subtracting mean 
def AverageRatingSub(ratingMatrix):
    return ratingMatrix.sub(ratingMatrix.mean(axis=1),axis=0).fillna(0)
#### Step-2- Calculating the Cosine Similarity  of the Normalized Utility Matrix
def cosineSimilarity(userID):
    CenteredMatrix=AverageRatingSub(UtilityMatrix)
    return   pd.DataFrame(cosine_similarity(CenteredMatrix.loc[[userID],:],
                                            CenteredMatrix.values).flatten(),
                                            index=UtilityMatrix.index,
                                            columns=['Similarity'])

#### Step-3- Predicting the Ratings of different Books of according to the ratings of other similar users.  
def RatingPredictionUserbased(userID,ratingMat):
    TopUserRatings=pd.DataFrame(columns=ratingMat.columns)
    SimilarMat=cosineSimilarity(userID) #Calculating the Similarity
    for index in SimilarMat.index:
        TopUserRatings=TopUserRatings.append(ratingMat.loc[[index],:],)
        TopUserRatings=TopUserRatings.replace(0,np.nan)
    TopUserRatings.fillna(TopUserRatings.mean(),inplace=True)
    return TopUserRatings

In [11]:
def UserBased(UserID,n):
    UserInitialRates=UtilityMatrix.loc[[UserID]]
    UserPredictedRatings=RatingPredictionUserbased(UserID,AverageRatingSub(UtilityMatrix)).loc[[UserID]]
    #Filter the Predicted ratings of Books that are not rated by the User(Books not read by the User)
    UserPredictedRatings=UserPredictedRatings.loc[:,
                        (UserInitialRates.isnull().sum(axis=0)>=1)]
    #Normalize the Centered and Ajusted Predicted Ratings to the range of 0-10 
    NormalizedPredictedRatings=pd.DataFrame(10*(UserPredictedRatings.loc[UserID,:]-UserPredictedRatings.loc[UserID,:].min())/(UserPredictedRatings.loc[UserID,:].max()-UserPredictedRatings.loc[UserID,:].min()))

    TopRecommendedBooks = pd.merge(left=FilteredBooks,right=NormalizedPredictedRatings,  
                                  how='inner', left_on='ISBN', 
                                  right_on='ISBN')
    TopRecommendedBooks.rename(columns={UserID: 'PredictedRating'},inplace=True)
    return TopRecommendedBooks.sort_values(by='PredictedRating',ascending=False).head(n)

### Test Case for User-based Recommendation

In [29]:
UserID=153662
UserBased(UserID,10)

,ISBN,BookTitle,BookAuthor,YearOfPublication,Publisher,BookDetails,PredictedRating
13438,0393049922,The Annotated Wizard of Oz: A Centennial Edition,L. Frank Baum,2000.0,W.W. Norton &amp; Company,The Annotated Wizard of Oz: A Centennial Editi...,10.0
6646,0060175907,Aphrodite: A Memoir of the Senses,Isabel Allende,1998.0,HarperCollins,"Aphrodite: A Memoir of the Senses,Isabel Allen...",10.0
14808,0892839007,A Life for God: The Mother Teresa Reader,Teresa,1995.0,St Anthony Messenger Pr,"A Life for God: The Mother Teresa Reader,Teres...",10.0
13595,1559210354,"Imperial Woman (Buck, Pearl S. Oriental Novels...",Pearl S. Buck,1991.0,Moyer Bell Ltd.,"Imperial Woman (Buck, Pearl S. Oriental Novels...",10.0
8100,0824819179,Noli Me Tangere (Shaps Library of Translations),Jose P. Rizal,1997.0,University of Hawaii Press,Noli Me Tangere (Shaps Library of Translations...,10.0
4812,0140074570,The Pool in the Desert (Penguin Short Fiction),Sara Jeannette Duncan,1985.0,Penguin USA,The Pool in the Desert (Penguin Short Fiction)...,10.0
14827,084466250X,The Road to Oz,Frank L. Baum,1986.0,Peter Smith Publisher Inc,"The Road to Oz,Frank L. Baum,1986.0,Peter Smit...",10.0
9011,0816708487,Emerald City of Oz,Frank L. Baum,1987.0,Troll Communications Llc,"Emerald City of Oz,Frank L. Baum,1987.0,Troll ...",10.0
4831,0399134204,Joy Luck Club,Amy Tan,1989.0,Putnam Publishing Group,"Joy Luck Club,Amy Tan,1989.0,Putnam Publishing...",10.0
14805,0892837438,Blessed Are You: Mother Teresa and the Beatitudes,Eileen Egan,1992.0,Servant Pubns,Blessed Are You: Mother Teresa and the Beatitu...,10.0


### Model-Based Recommendation 
##### I use the  Surprised Frame work of SVD (Matrix Factorization)

In [13]:
reader = Reader(rating_scale=(1, 10))
ratingsData = Dataset.load_from_df(MergedBooksRatings[['UserID', 'ISBN', 'BookRating']], reader)

In [14]:
#Building the Model 
SVDModel=SVD()
TrainData, TestData = train_test_split(ratingsData, test_size=.25)
SVDModel.fit(TrainData)

In [15]:
def ModelBasedSVD(user,nth):
    Rating = MergedBooksRatings
    Rating=Rating[Rating['UserID']!=user]
    Rating['est']=Rating['ISBN'].apply(lambda x:SVDModel.predict(user,x).est)
    Rating=Rating.drop_duplicates('ISBN')
    Rating=Rating.sort_values('est', ascending=False)
    return Rating.head(nth).drop(['UserID','BookRating'], axis=1)

### Test Cases of the SVD Model

In [16]:
ModelBasedSVD(UserID,10)

,ISBN,BookTitle,BookAuthor,YearOfPublication,Publisher,est
14355,0974408425,Cobo's Quest,Elizabeth A. Merz,2004.0,Quiet Storm Books,10.000000
12050,0451206770,A Deep Blue Farewell (Scotia MacKinnon Mysteries),Sharon Duncan,2002.0,Signet Book,10.000000
2390,0425181847,A Deadly Arrangement (Feng Shui Mysteries),Denise Osborne,2001.0,Prime Crime,10.000000
17149,0671491776,Luscher Color Test,Max Luscher,1982.0,Washington Square Press,10.000000
3431,0515124621,Then Came Heaven,LaVyrle Spencer,1999.0,Jove Books,9.970369
3324,0452281229,The Chisellers,Brendan O'Carroll,2000.0,Plume Books,9.969312
17521,0060175907,Aphrodite: A Memoir of the Senses,Isabel Allende,1998.0,HarperCollins,9.930339
6366,2070531430,"L'EpopÃ?Â©e du jazz, tome 2 : Au-delÃ?Â du bop",Franck Bergerot,1991.0,Gallimard,9.913964
1108,0330489070,How to Change Your Life,Rosie Milne,2002.0,Pan Macmillan,9.904548
5335,081257995X,The Eye of the World : Book One of 'The Wheel ...,Robert Jordan,2000.0,Tor Books,9.884431


### Content-Based Recommendation
This uses the combination of Book Title, Book Author, Year of Publication and Publisher to recommend books based on the ISBN of a particular Book

It Uses <b> Term Frequency - Inverse Document Frequency(TFIDF) </b>to determine the similary among the documents. 

In [17]:
#Importing TFIDF Sklearn library for Content based Recommendation
from sklearn.feature_extraction.text import TfidfVectorizer
#Importing the Linear_kernel for Cosine Similarity Calculation 
from sklearn.metrics.pairwise import linear_kernel

In [18]:
#Creating a new column by Collecting Books details from BookTitle, BookAuthor, YearOfPublication and Publisher
FilteredBooks['BookDetails'] = FilteredBooks[['BookTitle','BookAuthor','YearOfPublication','Publisher']].apply(
    lambda val: ','.join(val.dropna().astype(str)), axis=1)

In [19]:
#Defining the Term Frequency-Inverse Document Frequency(TF-IDF) Model
BooksTFIDF = TfidfVectorizer(encoding='utf-8', 
                             analyzer='word',ngram_range=(1,3) ,
                             stop_words='english')  
#Creating the TF-IDF Matrix
BooksTFIDFMatrix=BooksTFIDF.fit_transform(FilteredBooks['BookDetails'])

In [20]:
SimilarityMatrix = linear_kernel(BooksTFIDFMatrix, BooksTFIDFMatrix)

In [21]:
def ContentBasedRecommender(ISBN,n):
    sim=SimilarityMatrix[FilteredBooks[FilteredBooks.ISBN==ISBN].index]
    FlatListSimilarity = [x for sublist in sim for x in sublist]
    FilteredBooks.index = range(len(FilteredBooks))
    TopBookstoRecommend=pd.merge(right=pd.DataFrame(FlatListSimilarity,columns=["Similarity"]),left=FilteredBooks,  
                                  how='outer', left_index=True, right_index=True)
    TopBookstoRecommend=TopBookstoRecommend[FilteredBooks.ISBN!=ISBN].sort_values(
                    by='Similarity',ascending=False).drop_duplicates(subset=['ISBN',
                                                                 'BookTitle','BookAuthor',
                                                             'YearOfPublication'])
    return TopBookstoRecommend.head(n).drop('BookDetails',axis=1)

### Test Case for Content-based Recommendation System 

In [26]:
ISBN='0002005018'
ContentBasedRecommender(ISBN,10)

,ISBN,BookTitle,BookAuthor,YearOfPublication,Publisher,Similarity
8968,0060506067,Clara Callan : A Novel,Richard B. Wright,2002.0,HarperCollins,0.225076
7597,0843948647,Sleepeasy,T. M. Wright,2001.0,Leisure Books,0.133867
13548,0446526908,"Wait and See, Annie Lee : A Novel",Michelle Curry Wright,2001.0,Warner Books,0.082569
6027,0394297393,Callanetics,Callan Pinckney,1990.0,Sound Editions,0.057196
18578,0778011631,Where to Eat in Canada 01-02 (Where to Eat in ...,Anne Hardy,2001.0,Oberon Pr,0.056587
18411,0143013505,Unusual Heroes: Canada's Prime Ministers and F...,Shane Peacock,2002.0,Penguin Books Canada,0.050805
7246,0312135084,Henry and Clara,Thomas Mallon,1995.0,Picador USA,0.049515
12644,1590520319,"Out of the Ruins (Wright, Sally S. Ben Reese M...",Sally S. Wright,2003.0,Multnomah Publishers,0.048845
4382,0060812516,Uncle Tom's Children (Perennial Library),Richard Wright,1993.0,Perennial,0.047058
7470,0330281240,On the Black Hill,Bruce Chatwin,0.0,Pan Books Ltd,0.040587


### General Books Recommenders System 
This part mainly combines the top recommendations the three Models:
1. Memory-based Recommendation - particularly User-Based Recommender System
2. Model-based Recommendation -singular value decomposition(SVD) Model
3. Content-Based Recommendation -TF-IDF Model


Tries to satisfy and maximize the preferences of the User. 
The recommendation mainly combines the (n/3) results from Each to recommend n top books to the Users. 

In [27]:
def HybridBookRecommender(UserID,ISBN,n):
    FromContentBased = ContentBasedRecommender(ISBN,int(np.ceil(n/3))).drop(['Similarity'], axis=1)
    FromModelBased=ModelBasedSVD(UserID,int(np.round(n/3))).drop(['est'], axis=1)
    FromUserBased=UserBased(UserID,int(np.round(n/3))).drop(['BookDetails', 'PredictedRating'], axis=1)
    TopRecommendations=FromContentBased.append(FromModelBased, ignore_index = True)
    TopRecommendations=TopRecommendations.append(FromUserBased,ignore_index = True)
    return TopRecommendations.drop_duplicates()

### Test Case for Content-based Recommendation System 

In [28]:
UserID=189835
ISBN='1586420682'
n=10
HybridBookRecommender(UserID,ISBN,n)

,ISBN,BookTitle,BookAuthor,YearOfPublication,Publisher
0,031298670X,Elliott Roosevelt's Murder at the President's ...,Elliott Roosevelt,2003.0,St. Martin's Minotaur
1,0312274998,Murder at the President's Door: An Eleanor Roo...,Elliott Roosevelt,2001.0,Thomas Dunne Books
2,0312979193,Murder in the Lincoln Bedroom : An Eleanor Roo...,Elliott Roosevelt,2002.0,St. Martin's Minotaur
3,0394721713,There he goes again: Ronald Reagan's reign of ...,Ronald Reagan,1983.0,Pantheon Books
4,0312195516,The Red Tent (Bestselling Backlist),Anita Diamant,1998.0,Picador USA
5,0671729454,If There Be Thorns (Dollanganger),V.C. Andrews,1990.0,Pocket
6,0804114986,The Bonesetter's Daughter,Amy Tan,2002.0,Ballantine Books
7,096139210X,The Visual Display of Quantitative Information,Edward R. Tufte,1992.0,Graphics Press
8,0679451234,The Sibley Guide to Bird Life &amp; Behavior,David Allen Sibley,2001.0,Knopf
9,0451458222,"The Summer Tree (The Fionavar Tapestry, Book 1)",Guy Gavriel Kay,2001.0,Roc


# Thanks! End of The Code!